# Imports y obtención de archivos

In [1]:
import pandas as pd
import math

pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

# Inicialización de los dataframes

*contiene código importante para algunas de las preguntas*

In [5]:
ps_apps = pd.read_csv("googleplaystore.csv", skiprows=[10473], usecols=["App", "Category"])

# Saco las apps duplicadas, me quedo con la primera línea (por ninguna razón en particular)
ps_apps = ps_apps.drop_duplicates(subset=["App"], keep="first")

ps_apps["Category"] = ps_apps["Category"].astype("category")

# Pregunta #37
[*link al notebook general*](https://colab.research.google.com/drive/1MpsnwNHS9Uyfj3PH0kcRIPYK6PhC-bGQ?usp=drive_link)

> *Devolver las categorías que utilicen más caracteres distintos para sus apps, y las que menos caracteres distintos usan. ¿Cuáles son los caracteres que se comparten entre ambas?*

**Respuesta:**
- *Con más*: Family (211 caracteres)
- *Con menos*: Beauty (39 caracteres)
- *Compartidos*: ver salida (Son todos los de *Beauty*)

**Notas:**
- Me pareció más sencillo definir la función de contar caracteres aquí que meter las librerías de nlp *solamente* para este pedacito de esta pregunta (además de que me convenía empezar de una a trabajar con sets para la intersección).
- Decidi contar mayúsculas y minúsculas como caracteres iguales
- Me fijé de que ambos valores fueran únicos (además el mayor le saca bastante al segundo)

In [6]:
# Esta función me parece un poco bruta, pero sirve
def charset(series):
  cset = set({})
  for item in series:
    for letter in item:
      cset.add(letter.lower())
  return cset

df37 = (ps_apps.groupby("Category")
            .agg({"App": charset})
            .reset_index()
            .rename(columns={"App": "Characters"}))
df37["Character_Count"] = df37.apply(lambda row: len(row["Characters"]), axis=1)

top = df37.nlargest(1, "Character_Count")
bottom = df37.nsmallest(1, "Character_Count")
shared = top.iloc[0]["Characters"].intersection(bottom.iloc[0]["Characters"])

# A partir de aquí es para que la salida me quede más bonita
topcat = top.iloc[0]["Category"]
bottomcat = bottom.iloc[0]["Category"]

topcount = top.iloc[0]["Character_Count"]
bottomcount = bottom.iloc[0]["Character_Count"]

print(f"==> Categoría con más caracteres únicos: {topcat} ({topcount} caracteres)")
print(f"==> Categoría con menos caracteres únicos: {bottomcat} ({bottomcount} caracteres)")
print(f"==> {len(shared)} caracteres compartidos: {shared}")

==> Categoría con más caracteres únicos: FAMILY (211 caracteres)
==> Categoría con menos caracteres únicos: BEAUTY (39 caracteres)
==> 39 caracteres compartidos: {'1', 'l', '0', '&', '/', 't', '2', 'h', 'w', 'g', 'i', 'v', '+', 'x', ')', 'm', ':', 'j', '3', 'c', 'd', 'k', 's', 'o', 'p', 'e', 'r', '(', 'b', 'u', ',', 'y', '8', ' ', 'n', 'z', 'f', 'a', '-'}
